In [7]:
import pandas as pd
import os
import datetime
import numpy as np
import json
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/temp'

In [8]:
dict_xx=json.load(open("./dict_XX_prodcut.json","r"))
dict_yy=json.load(open("./dict_YY_prodcut.json","r"))

In [9]:
df_yy=pd.read_csv("./data/Antai_AE_round1_test_20190626.csv",dtype=str,usecols=['buyer_country_id','item_id'])
print(df_yy.shape)
df_xx=pd.read_csv("./data/Antai_AE_round1_train_20190626.zip",compression="zip",dtype=str,usecols=['buyer_country_id','item_id'])
df_xx=df_xx[df_xx['buyer_country_id']=="yy"]
df_yy=df_yy.append(df_xx)
del df_xx
gc.collect()


df_most_popular_30_items=df_yy.groupby("item_id")['buyer_country_id'].count().to_frame().reset_index().sort_values("buyer_country_id",ascending=False)
most_popular_30_items=df_most_popular_30_items.head(30)['item_id'].tolist()

print(df_yy.shape)
del df_yy
gc.collect()

(166832, 2)
(2399699, 2)


21

In [47]:
df_most_popular_30_items['buyer_country_id'].sum()

2399699

In [10]:
df_YY_total=pd.DataFrame()
for key,value in dict_yy.items():
    df=pd.DataFrame(value,index=["prob_yy"]).T
    df['hold_item']=key
    df=df.reset_index().rename(columns={"index":"pred_item"})
    
    df_YY_total=df_YY_total.append(df)
df_YY_total.shape

(122630, 3)

In [11]:
df_XX_total=pd.DataFrame()
for key,value in dict_xx.items():
    df=pd.DataFrame(value,index=["prob_xx"]).T
    df['hold_item']=key
    df=df.reset_index().rename(columns={"index":"pred_item"})
    
    df_XX_total=df_XX_total.append(df)
df_XX_total.shape

(1161314, 3)

In [12]:
df_hold_pred_items=pd.merge(df_XX_total,df_YY_total,on=['hold_item','pred_item'],how="outer")
df_hold_pred_items=df_hold_pred_items.fillna(0)
# df_hold_pred_items['weighted_prob']=df_hold_pred_items['prob_xx']*0.4+df_hold_pred_items['prob_yy']*0.6
df_hold_pred_items['weighted_prob']=np.where(df_hold_pred_items['prob_xx']==0,df_hold_pred_items['prob_yy'],
                                            np.where(df_hold_pred_items['prob_yy']==0,df_hold_pred_items['prob_xx']*0.95,
                                                    df_hold_pred_items['prob_yy']*0.85+df_hold_pred_items['prob_xx']*0.15)
                                            )
# df_hold_pred_items['prob_xx']*0.4+df_hold_pred_items['prob_yy']*0.6

df_hold_pred_items=df_hold_pred_items.sort_values(['hold_item','weighted_prob'],ascending=[True,False]).reset_index()
del df_hold_pred_items['index']
df_hold_pred_items.shape

(1275788, 5)

In [14]:
df_test=pd.read_csv("./data/Antai_AE_round1_test_20190626.csv",dtype=str)
df_test['irank']=df_test['irank'].astype(int)
df_test=df_test.sort_values(["buyer_admin_id","create_order_time","irank"])
df_test.shape

(166832, 5)

In [15]:
df_temp=df_test[['buyer_admin_id','create_order_time']].drop_duplicates()
new_rank=[]
for buyer_id,df_group in df_temp.groupby("buyer_admin_id"):
    new_rank=new_rank+[x+1 for x in range(len(df_group))]
        
df_temp['new_rank']=new_rank
df_test=pd.merge(df_test,df_temp,on=['buyer_admin_id','create_order_time'])

In [16]:
df_dict_prob=df_hold_pred_items[['pred_item','hold_item','weighted_prob']]
print(df_dict_prob.shape)
print(df_dict_prob[['pred_item','hold_item']].drop_duplicates().shape)
df_dict_prob=df_dict_prob.rename(columns={"hold_item":"item_id"})

(1275788, 3)
(1275788, 2)


In [19]:
df_test['date_time']=pd.to_datetime(df_test['create_order_time'],format="%Y-%m-%d %H:%M:%S")
df_test['date']=df_test['date_time'].apply(lambda x: x.date())
df_test.head(10)

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,new_rank,date_time,date
0,yy,1000334,7610114,2018-08-16 09:23:33,18,1,2018-08-16 09:23:33,2018-08-16
1,yy,1000334,7610114,2018-08-16 09:23:33,19,1,2018-08-16 09:23:33,2018-08-16
2,yy,1000334,9559864,2018-08-16 09:48:57,16,2,2018-08-16 09:48:57,2018-08-16
3,yy,1000334,10908604,2018-08-16 09:48:57,17,2,2018-08-16 09:48:57,2018-08-16
4,yy,1000334,6271572,2018-08-16 09:52:21,15,3,2018-08-16 09:52:21,2018-08-16
5,yy,1000334,11549978,2018-08-17 09:25:11,14,4,2018-08-17 09:25:11,2018-08-17
6,yy,1000334,4326555,2018-08-21 04:13:38,13,5,2018-08-21 04:13:38,2018-08-21
7,yy,1000334,1313773,2018-08-27 02:52:21,9,6,2018-08-27 02:52:21,2018-08-27
8,yy,1000334,838639,2018-08-27 02:52:21,10,6,2018-08-27 02:52:21,2018-08-27
9,yy,1000334,4982387,2018-08-27 02:52:21,11,6,2018-08-27 02:52:21,2018-08-27


In [20]:
df_test_iter=df_test[["buyer_admin_id",'new_rank',"item_id",'date']]
df_test_iter['buyer_admin_id'].nunique()

11398

In [54]:
col_30=[x+1 for x in range(30)]
df_output=pd.DataFrame(columns=['buyer_id']+col_30)
i=0
i_fill=0
for buyer_id,df_group in df_test_iter.groupby(["buyer_admin_id"]):
    purchased_items=df_group['item_id'].tolist()
    last_item=purchased_items[-1]
    df_date_max=df_group['date'].max()
    # break
    df_group=df_group[df_group['date']>=(df_date_max-datetime.timedelta(days=7))]
    df_group['days_delta']=df_group['date'].apply(lambda x: (7-(df_date_max-x).days))
    # break
    df_group=pd.merge(df_group,df_dict_prob,on="item_id",how="left")
    df_group=df_group.fillna(0)
    df_group=df_group[df_group['weighted_prob']!=0]
    
    df_group['final_score']=df_group['weighted_prob']*df_group['days_delta']*df_group['days_delta']

    df_group=df_group.groupby("pred_item")['final_score'].sum().to_frame().reset_index()
    
    df_group['final_score']=np.where(df_group['pred_item'].isin(purchased_items),df_group['final_score']*0.01,df_group['final_score'])
    df_group=df_group.sort_values("final_score",ascending=False).head(30)
    
    df_list_30=df_group['pred_item'].tolist()
    if len(df_list_30)<=20:
        i_fill+=1
    df_list_30=df_list_30+most_popular_30_items[:30-len(df_list_30)]

    df=pd.DataFrame({"result":df_list_30},index=col_30).T
    df['buyer_id']=buyer_id
    
    df_output=df_output.append(df)
    i+=1
    if i%100==2:
        print(datetime.datetime.now(),i)

print(i,i_fill)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


2019-08-11 15:51:01.104469 2
2019-08-11 15:51:23.369738 102
2019-08-11 15:51:45.879314 202
2019-08-11 15:52:08.452527 302
2019-08-11 15:52:30.956277 402
2019-08-11 15:52:53.399148 502
2019-08-11 15:53:16.089655 602
2019-08-11 15:53:38.513950 702
2019-08-11 15:54:00.989134 802
2019-08-11 15:54:23.587810 902
2019-08-11 15:54:46.154620 1002
2019-08-11 15:55:08.473112 1102
2019-08-11 15:55:30.900415 1202
2019-08-11 15:55:53.547416 1302
2019-08-11 15:56:16.108890 1402
2019-08-11 15:56:38.748398 1502
2019-08-11 15:57:01.078289 1602
2019-08-11 15:57:23.694475 1702
2019-08-11 15:57:46.403723 1802
2019-08-11 15:58:09.009830 1902
2019-08-11 15:58:31.344580 2002
2019-08-11 15:58:53.715688 2102
2019-08-11 15:59:16.159068 2202
2019-08-11 15:59:38.275840 2302
2019-08-11 16:00:00.692297 2402
2019-08-11 16:00:23.311006 2502
2019-08-11 16:00:45.990215 2602
2019-08-11 16:01:08.476624 2702
2019-08-11 16:01:31.188951 2802
2019-08-11 16:01:53.708460 2902
2019-08-11 16:02:16.422459 3002
2019-08-11 16:02:39.

In [59]:
df_output.to_csv("./outputV3_JL_"+str(datetime.datetime.now().date()).replace("-","")+".csv",index=False,header=False)

In [58]:
df_output.shape

(11398, 31)

In [56]:
for col in df_output.columns.tolist():
    print(col,df_output[col].apply(lambda x: type(x)).unique())

buyer_id [<class 'str'>]
1 [<class 'str'>]
2 [<class 'str'>]
3 [<class 'str'>]
4 [<class 'str'>]
5 [<class 'str'>]
6 [<class 'str'>]
7 [<class 'str'>]
8 [<class 'str'>]
9 [<class 'str'>]
10 [<class 'str'>]
11 [<class 'str'>]
12 [<class 'str'>]
13 [<class 'str'>]
14 [<class 'str'>]
15 [<class 'str'>]
16 [<class 'str'>]
17 [<class 'str'>]
18 [<class 'str'>]
19 [<class 'str'>]
20 [<class 'str'>]
21 [<class 'str'>]
22 [<class 'str'>]
23 [<class 'str'>]
24 [<class 'str'>]
25 [<class 'str'>]
26 [<class 'str'>]
27 [<class 'str'>]
28 [<class 'str'>]
29 [<class 'str'>]
30 [<class 'str'>]


In [57]:
datetime.datetime.now()

datetime.datetime(2019, 8, 11, 16, 35, 0, 828111)